In [1]:
import os
from tqdm.notebook import tqdm

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from datasets import load_dataset

from fairscale.nn.model_parallel.initialize import (
    initialize_model_parallel,
    model_parallel_is_initialized,
)
from llama.tokenizer import Tokenizer

from mixture_of_depths.routing_transformer import ModelArgs, MoDTransformer

In [2]:
if not torch.distributed.is_initialized():
    # torch.distributed.init_process_group("nccl")
    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '12345'
    torch.distributed.init_process_group(backend='gloo', rank=0, world_size=1)
if not model_parallel_is_initialized():
    model_parallel_size = int(os.environ.get("WORLD_SIZE", 1))
    initialize_model_parallel(model_parallel_size)

local_rank = int(os.environ.get("LOCAL_RANK", 0))
torch.cuda.set_device(local_rank)
torch.manual_seed(42)
torch.set_default_tensor_type(torch.cuda.FloatTensor)

> initializing model parallel with size 1
> initializing ddp with size 1
> initializing pipeline with size 1


c:\Users\Shivaen\anaconda3\envs\MoD\lib\site-packages\torch\__init__.py:696: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\tensor\python_tensor.cpp:453.)
  _C._set_default_tensor_type(t)


## Load Data

In [3]:
dataset = load_dataset(
    'allenai/c4', data_files={'validation': 'en/c4-validation.00000-of-00008.json.gz'}, split='validation'
)

In [4]:
dataset

Dataset({
    features: ['text', 'timestamp', 'url'],
    num_rows: 45576
})

In [5]:
tokenizer = Tokenizer("../llama/tokenizer.model")
def collate_fn(batch):
    bsz = len(batch)
    tokenized_texts = [tokenizer.encode(x['text'], bos=True, eos=True) for x in batch]
    max_text_len = max(len(t) for t in tokenized_texts)

    pad_id = tokenizer.eos_id
    tokens = torch.full((bsz, min(2048, max_text_len)), pad_id, dtype=torch.long)
    for k, t in enumerate(tokenized_texts):
        tokens[k, : len(t)] = torch.tensor(t, dtype=torch.long)[:2048]
    
    return tokens[:,:-1], tokens[:,1:]

dataloader = DataLoader(
    dataset,
    batch_size=4,
    collate_fn=collate_fn,
)

In [6]:
next(iter(dataloader))

(tensor([[    1,   450,  6114,  ...,     2,     2,     2],
         [    1,  4473,  6751,  ...,     2,     2,     2],
         [    1, 26871, 17101,  ...,  4404,   331, 29889],
         [    1,   382,  5348,  ...,     2,     2,     2]]),
 tensor([[  450,  6114,  1058,  ...,     2,     2,     2],
         [ 4473,  6751,  1788,  ...,     2,     2,     2],
         [26871, 17101,   379,  ...,   331, 29889,     2],
         [  382,  5348,   399,  ...,     2,     2,     2]]))

## Train

In [7]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [8]:
class MoDLlamaTrainer():
    def __init__(self, model, dataloader):
        self.model = model
        self.dataloader = dataloader

    def train(
        self,
        epochs=10,
        lr=1e-5,
        model_path="./models/MoDLlama.pt",
        log_path="MoDLlama_log.txt",
        causal_loss=False,
        log_steps=1000,
    ):
        min_loss = float("inf")
        criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_id)
        optimizer = torch.optim.AdamW(self.model.parameters(), lr)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)

        for epoch in range(epochs):
            self.model.train()
            running_loss = 0.0
            for i, (inputs, targets) in enumerate(tqdm(self.dataloader, desc=f"Epoch: {epoch}")):
                optimizer.zero_grad()

                outputs = self.model(inputs, start_pos=0)

                loss = criterion(outputs.permute(0, 2, 1), targets)
                if causal_loss:
                    loss += criterion()

                loss.backward()
                optimizer.step()
                scheduler.step()
                nn.utils.clip_grad_norm_(self.model.parameters(), 0.5)

                running_loss += loss.detach().cpu().item()

                if (i+1) % log_steps == 0:
                    avg_loss = running_loss / (i+1)
                    print(f"Loss as step {i+1}: {avg_loss}")

            epoch_loss = running_loss / len(self.dataloader)
            if min_loss > epoch_loss:
                torch.save(self.model.state_dict(), model_path)
                min_loss = epoch_loss
            
            print(f"Epoch {epoch+1}/{epochs} - Loss: {epoch_loss}")
            with open(log_path, 'w') as f:
                f.write(f"Epoch {epoch+1}/{epochs} - Loss: {epoch_loss}")
                f.write("\n")

With MoD

In [9]:
model_params = ModelArgs(
    dim=512,
    n_layers=6,
    n_heads=8,
    vocab_size=tokenizer.n_words,
    routing=True,
)
model = MoDTransformer(model_params)

In [10]:
count_parameters(model)

53222401

In [11]:
trainer = MoDLlamaTrainer(model, dataloader)

In [12]:
%%time
trainer.train(
    epochs=1,
)

Epoch: 0:   0%|          | 0/11394 [00:00<?, ?it/s]

Loss as step 1000: 7.283473222732544
Loss as step 2000: 6.11342502105236
Loss as step 3000: 5.442867278575897
Loss as step 4000: 5.044269517302513
Loss as step 5000: 4.791520118331909
Loss as step 6000: 4.601423125704129
Loss as step 7000: 4.462108738575663
Loss as step 8000: 4.34014173284173
Loss as step 9000: 4.242263604534997
Loss as step 10000: 4.16241569558382
Loss as step 11000: 4.08866784259406
Epoch 1/1 - Loss: 4.058898564135796
CPU times: total: 5min 48s
Wall time: 28min 4s


Baseline

In [9]:
model_params = ModelArgs(
    dim=512,
    n_layers=6,
    n_heads=8,
    vocab_size=tokenizer.n_words,
    routing=False,
)
model = MoDTransformer(model_params)

In [10]:
count_parameters(model)

53221888

In [11]:
trainer = MoDLlamaTrainer(model, dataloader)

In [12]:
%%time
trainer.train(
    epochs=1,
)

Epoch: 0:   0%|          | 0/11394 [00:00<?, ?it/s]

Loss as step 1000: 7.209186413764954
Loss as step 2000: 6.076255900025368
Loss as step 3000: 5.418029305934906
Loss as step 4000: 5.024323759019375
Loss as step 5000: 4.773851863384246
Loss as step 6000: 4.5845877949992815
Loss as step 7000: 4.4457164600746975
Loss as step 8000: 4.323944765463471
Loss as step 9000: 4.226143040763008
Loss as step 10000: 4.146285073900223
Loss as step 11000: 4.072505737889896
Epoch 1/1 - Loss: 4.042734246672884
CPU times: total: 4min 41s
Wall time: 32min 33s
